## Ce jupiter traitera l'insertion dans postgresql

### Installation et mise à jour des bibliothèques

In [ ]:
pip install --upgrade pandas

In [ ]:
pip install psycopg2

In [ ]:
import pandas as pd

In [ ]:
#immo = pd.read_csv('C:/Users/suzyg/OneDrive/Documents/Ecoles/Epitech/T-DAT-902/Données/france_total_real_estate_sales_2022.csv')

#codes_departements_outre_mer = [971, 972, 973, 974, 976]
#donnees_metropolitaines = immo[immo['Code departement'].isin(codes_departements_outre_mer)]
#print(donnees_metropolitaines)

### Chargement et nettoyage des données

#### Chargement

In [ ]:
#Chargement des fichiers
brutes_communes = pd.read_csv('./data/communes-departement-region.csv', dtype=str)
brutes_ecoles = pd.read_csv('./data/fr-en-annuaire-education.csv', sep=';', dtype=str)

In [ ]:
#Affichage des données
print("************************************** COMMUNES **************************************")

brutes_communes['code_postal_commune'] = brutes_communes['code_commune_INSEE'] +'-'+ brutes_communes['code_postal']
code_commune_unique=brutes_communes[['code_postal_commune', 'code_commune_INSEE']].drop_duplicates()
print(brutes_communes)

print("************************************** ECOLES **************************************")

brutes_ecoles['code_postal_commune'] = brutes_ecoles['code_postal'] +'-'+ brutes_ecoles['code_commune']
brutes_ecoles = pd.merge(brutes_ecoles, code_commune_unique, how='left')
brutes_ecoles = brutes_ecoles.dropna(subset='code_commune_INSEE')
print(brutes_ecoles)

colonnes_communes = ['code_commune_INSEE', 'nom_commune', 'code_postal', 'code_postal_commune', 'latitude', 'longitude', 'code_departement', 'nom_departement', 'code_region', 'nom_region']
colonnes_ecoles = ['identifiant_de_l_etablissement', 'nom_etablissement', 'type_etablissement', 'statut_public_prive', 'adresse_1','code_postal_commune']

#### Nettoyage

In [ ]:
#fonction pour nettoyer
def nettoyage (fichier, colonnes):

    #On ne sélectionne que les données pertinentes
    pertinentes = fichier.loc[:, colonnes]
    
    #Suppression des lignes contenant dont les id contiennent des valeurs NaN et on remplace le reste par des espaces
    sans_nan = pertinentes.dropna(subset=colonnes[0])
    sans_nan[colonnes]=sans_nan[colonnes].fillna('')
    
    #Supprimes les doublons
    df= sans_nan.drop_duplicates()

    #conversion en tuples
    #tuples = list(sans_doublons.itertuples(index=False, name=None))
    #tpls = [tuple(x) for x in df.to_numpy()]
    
    # dataframe columns with Comma-separated
    #cols = ','.join(list(df.columns))
    return df
    
print("************************************** COMMUNES **************************************")
communes = nettoyage(brutes_communes, colonnes_communes)
print(communes)
#print(communes[:5])

print("************************************** ECOLES **************************************")
ecoles = nettoyage (brutes_ecoles, colonnes_ecoles)
print(ecoles)
#print(ecoles[:5])

#********************************************************************************************
print(ecoles)

### Connexion au serveur de base de données

In [ ]:
import sys
import psycopg2
from psycopg2 import OperationalError, errorcodes, errors
import psycopg2.extras as extras

params= {
    'host':'localhost',
    'user':'postgres',
    'password':'pass',
    'port':'5433'
}


#Gestion des erreurs
def erreur_psycopg2(err):
    #on récupère les détails de l'exception
    err_type, err_obj, traceback= sys.exc_info()

    #Récupération de la ligne qui a causé l'erreur
    line_n=traceback.tb_lineno

    #Affichage de l'erreur
    print("\npsycopg2 ERROR : ", err, "sur la ligne: ", line_n)
    print("psycopg2 traceback: ", traceback,"--type: ", err_type)
    
    # psycopg2 extensions.Diagnostics object attribute
    print ("\nextensions.Diagnostics:", err.diag)
    
    # print the pgcode and pgerror exceptions
    print ("pgerror:", err.pgerror)
    print ("pgcode:", err.pgcode, "\n")

#Connexion au serveur de base de données
def connect(params):
    conn = None
    try:
        print('Connexion à PostgreSQL...........')
        conn = psycopg2.connect(**params)
        print("Connexion réussie..................")
    except OperationalError as err:
        erreur_psycopg2(err)
        # On réinitialise la connexion en cas d'erreur
        conn = None
    
    return conn

### Création de la base de données

In [ ]:
conn= connect(params)
conn.autocommit = True

if conn!= None:
    try:
        cursor = conn.cursor()
        #Suppression de la base de données si elle existe déjà
        cursor.execute("DROP DATABASE IF EXISTS homepedia;")
        
        #Création de la base de données
        cursor.execute("CREATE DATABASE homepedia")
        print("Base de données créée avec succès !!!")
        cursor.close()
        conn.close()
        
    except OperationalError as err:
        erreur_psycopg2(err)
        conn = None

### Création des tables

In [ ]:
params2= {
    'host':'localhost',
    'database':'homepedia',
    'user':'postgres',
    'password':'pass',
    'port':'5433'
}
TABLES = {}
TABLES['communes'] = (
    "CREATE TABLE communes ("
    "  num_row SERIAL PRIMARY KEY,"
    "  code_commune_INSEE text NOT NULL,"
    "  nom_commune text NOT NULL,"
    "  code_postal text NOT NULL,"
    "  code_postal_commune text UNIQUE,"
    "  latitude text,"
    "  longitude text,"
    "  code_departement text,"
    "  nom_departement text NOT NULL,"
    "  code_region text,"
    "  nom_region text NOT NULL"
    ")")

#TABLES['code_postal_commune_index']=( "CREATE UNIQUE INDEX code_commune_index ON communes(code_postal_commune)")

TABLES['ecoles'] = (
    "CREATE TABLE ecoles ("
    "num_row SERIAL PRIMARY KEY,"
    "identifiant_de_l_etablissement text NOT NULL,"
    "nom_etablissement text NOT NULL,"
    "type_etablissement text NOT NULL,"
    "statut_public_prive text NOT NULL,"
    "adresse_1 text,"
    "code_postal_commune text REFERENCES communes (code_postal_commune) ON DELETE CASCADE"
    ")")

conn= connect(params2)
conn.autocommit = True

for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        cursor = conn.cursor()
        
        cursor.execute("DROP TABLE IF EXISTS {} CASCADE".format(table_name))
        cursor.execute(table_description)
        print("Creation de la table {} avec succès: ".format(table_name))
        
        cursor.close()
    except OperationalError as err:
        erreur_psycopg2(err)
        conn = None
#conn.close()

### Insertion des données dans la base de données

In [ ]:
#Création d'une fonction pour l'insertion des données du dataframe
def execute_many(conn, df, table):
    # Creation d'une liste de tuplets à partir du dataframe
    tpls = [tuple(x) for x in df.to_numpy()]
    
    # dataframe columns with Comma-separated
    cols = ','.join(list(df.columns))
    
    if table == 'communes':
       sql = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    elif table =='ecoles':
       sql = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s,%%s)" %(table, cols)
        
    cursor = conn.cursor()
    try:
        cursor.executemany(sql, tpls)
        conn.commit()
        print("Données insérées avec succès...")
    except (Exception, psycopg2.DatabaseError) as err:
        erreur_psycopg2(err)
        cursor.close()

execute_many(conn, communes, 'communes')
execute_many(conn, ecoles, 'ecoles')
conn.close()